In [48]:
import json
from urllib.parse import urlparse

def extract_fqdn_and_date(file_paths: list):
    """
    Extracts the FQDN from the 'url' field and the 'date' from the provided JSON lines
    in multiple files.

    Args:
        file_paths (list): List of paths to files containing JSON entries.

    Returns:
        list: A list of tuples containing the FQDN and date for each entry from all files.
    """
    results = []
    urls = []

    for file_path in file_paths:
        with open(file_path, 'r') as file:
            # Read each line of the JSONL file
            for line in file:
                entry = json.loads(line)
                # Extract the FQDN from the 'url'
                url = entry.get("url", "")
                parsed_url = urlparse(url)
                urls.append(url)
                fqdn = parsed_url.netloc  # Get the FQDN part from the URL

                # Extract the 'date'
                date = entry.get("date", "")

                # Append the FQDN and date to the results
                results.append((fqdn, date))

    return results, urls

In [49]:
# File paths
file_path1 = r"C:\Users\ricewater\Documents\CTIDownloads\malpedia_20220718\malpedia_20220718\malpedia-db_2022-07-18_downloader.jsonl"
file_path2 = r"C:\Users\ricewater\Documents\CTIDownloads\20241204_malpedia_downloads\20241204_downloads.jsonl"

# Extract FQDNs and dates from both files
malpedia_fqdn_and_dates, malpedia_urls = extract_fqdn_and_date([file_path1, file_path2])

# Print the count of unique FQDNs across both files
print(len(set(fqdn for fqdn, date in malpedia_fqdn_and_dates)))
print("Total unique URLs:", len(set(urls)))

2002
Total unique URLs: 15695


In [40]:
import json
from urllib.parse import urlparse

def extract_successful_urls(file_path: str):
    """
    Extracts the FQDN from the 'url' field only for entries where the 'download_status' is 200 (successful).

    Args:
        file_path (str): Path to the file containing JSON entries.

    Returns:
        list: A list of FQDNs for entries with a successful download status.
    """
    successful_fqdn = []
    successful_urls = []

    with open(file_path, 'r') as file:
        # Read each line of the JSONL file
        for line in file:
            entry = json.loads(line)

            # Check if 'download_status' is 200
            if entry.get('download_status') == 200:
                # Extract the FQDN from the 'url'
                url = entry.get("url", "")
                parsed_url = urlparse(url)
                successful_urls.append(url)
                fqdn = parsed_url.netloc  # Get the FQDN part from the URL

                # Append the FQDN to the list
                if fqdn:
                    successful_fqdn.append(fqdn)

    return successful_urls, successful_fqdn

In [42]:
# Example Usage for MITRE
file_path = r"C:\Users\ricewater\Documents\CTIDownloads\downloads\downloads\20241008_downloads.jsonl" 
mitre_successful_urls, mitre_successful_fqdns = extract_successful_urls(file_path)
print("FQDNS:", len(set(mitre_successful_fqdns)))
print("URLS:" , len(set(mitre_successful_urls)))
# Display the results
#for fqdn in successful_fqdns:
#    print(f"FQDN: {fqdn}")

FQDNS: 179
URLS: 787


In [54]:
import pandas as pd
file_path_1 = r"C:\Users\ricewater\Documents\CTITTP\ATTACK Excel sheets\enterprise-attack-v15.1-groups.xlsx"

# Step 2: Read the "Citations" sheet and extract the "URL" column from each spreadsheet
df1 = pd.read_excel(file_path_1, sheet_name='citations')['url'].dropna()
print(len(df1), df1.nunique())
mitre_successful_urls = df1

mitre_successful_fqdns = []

for url in mitre_successful_urls:
                parsed_url = urlparse(url)
                fqdn = parsed_url.netloc  # Get the FQDN part from the URL
                if fqdn:
                    mitre_successful_fqdns.append(fqdn)

print(len(set(mitre_successful_fqdns)))
print(set(mitre_successful_fqdns))

1372 905
213
{'labs.sentinelone.com', 'threatconnect.com', 'geminiadvisory.io', 'blog.scilabs.mx', 'www.reuters.com', 'www.intezer.com', 'attack.mitre.org', 'www.volexity.com', 'docs.microsoft.com', 'www.mandiant.com', 'cloudblogs.microsoft.com', 'www.cisa.gov', 'www.malwarebytes.com', 'download.ahnlab.com', 'unit42.paloaltonetworks.com', 'www.rapid7.com', 'www.threatminer.org', 'www.ncsc.gov.uk', 'usa.kaspersky.com', 'blog.qualys.com', 'cyberscoop.com', 'www.uptycs.com', 'www.securityweek.com', 'pylos.co', 'objective-see.com', 'www.slideshare.net', 'blog.cylance.com', 'blog.alyac.co.kr', 'securityaffairs.co', 'threatpost.com', '2017-2021.state.gov', 'dragos.com', 'vb2020.vblocalhost.com', 'blogs.blackberry.com', 'vblocalhost.com', 'www.darkreading.com', 'insight-jp.nttsecurity.com', 'www.lacework.com', 'www.talent-jump.com', 'www.wired.com', 'www.accenture.com', 'cdn-cybersecurity.att.com', 'cycraft.com', 'blog.certfa.com', 'news.sophos.com', 'home.treasury.gov', 'www.f-secure.com', '

In [55]:
union_URLS = set(mitre_successful_urls).union(set(urls))
print(len(union_URLS))

16600


In [56]:
from collections import Counter

def compare_fqdns(malpedia_fqdn_and_dates: list, mitre_successful_fqdns: list):
    """
    Compares two sets of FQDNs, returning the overlap, A-B, B-A, and their counts.

    Args:
        fqdn_and_dates (list): A list of tuples containing FQDN and date.
        successful_fqdns (list): A list of FQDNs with successful download statuses.

    Returns:
        dict: A dictionary with the overlap, A-B, B-A, and counts for each.
    """
    # Extract unique FQDNs from both lists
    unique_fqdn_dates = set(fqdn for fqdn, date in malpedia_fqdn_and_dates)  # Unique FQDNs from fqdn_and_dates
    unique_successful_fqdns = set(mitre_successful_fqdns)  # Unique FQDNs from successful_fqdns

    # Find union
    union_set = unique_fqdn_dates.union(unique_successful_fqdns)
    
    # Find the overlap
    overlap = unique_fqdn_dates.intersection(unique_successful_fqdns)

    # Find A - B (FQDNs in fqdn_and_dates but not in successful_fqdns)
    a_minus_b = unique_fqdn_dates.difference(unique_successful_fqdns)

    # Find B - A (FQDNs in successful_fqdns but not in fqdn_and_dates)
    b_minus_a = unique_successful_fqdns.difference(unique_fqdn_dates)

    #Count the frequencies of FQDNs in Malpedia and MITRE lists
    malpedia_fqdn_counter = Counter(fqdn for fqdn, date in malpedia_fqdn_and_dates)
    mitre_fqdn_counter = Counter(mitre_successful_fqdns)

    # Get the top 5 FQDNs by frequency in Malpedia, MITRE, and overlap
    top_malpedia_fqdns = malpedia_fqdn_counter.most_common(10)
    top_mitre_fqdns = mitre_fqdn_counter.most_common(10)
 

    # Return the results along with the counts
    return {
        'union':union_set,
        'overlap': overlap,
        'A-B': a_minus_b,
        'B-A': b_minus_a,
        'union_count':len(union_set),
        'overlap_count': len(overlap),
        'A-B_count': len(a_minus_b),
        'B-A_count': len(b_minus_a),
        'top_malpedia_fqdns': top_malpedia_fqdns,
        'top_mitre_fqdns': top_mitre_fqdns,
    }


In [57]:
# Compare the FQDNs
comparison_results = compare_fqdns(malpedia_fqdn_and_dates, mitre_successful_fqdns)

# Display the results
print(f"Union (A  U B) Count: {comparison_results['union_count']}")
print(f"Overlap (A ∩ B): {comparison_results['overlap']}, Count: {comparison_results['overlap_count']}")
print(f"A - B (In Malpedia but not in MITRE) Count: {comparison_results['A-B_count']}")
print(f"B - A (In MITRE but not in Malpedia): {comparison_results['B-A']}, Count: {comparison_results['B-A_count']}")

# Print the top FQDNs by frequency
print("\nTop FQDNs in Malpedia (by frequency):")
for fqdn, count in comparison_results['top_malpedia_fqdns']:
    print(f"{fqdn}: {count} times")

print("\nTop FQDNs in MITRE (by frequency):")
for fqdn, count in comparison_results['top_mitre_fqdns']:
    print(f"{fqdn}: {count} times")




Union (A  U B) Count: 2024
Overlap (A ∩ B): {'labs.sentinelone.com', 'threatconnect.com', 'geminiadvisory.io', 'blog.scilabs.mx', 'www.reuters.com', 'www.intezer.com', 'attack.mitre.org', 'www.volexity.com', 'docs.microsoft.com', 'www.mandiant.com', 'cloudblogs.microsoft.com', 'www.cisa.gov', 'www.malwarebytes.com', 'download.ahnlab.com', 'unit42.paloaltonetworks.com', 'www.rapid7.com', 'www.ncsc.gov.uk', 'usa.kaspersky.com', 'blog.qualys.com', 'cyberscoop.com', 'www.uptycs.com', 'www.securityweek.com', 'pylos.co', 'objective-see.com', 'www.slideshare.net', 'blog.cylance.com', 'blog.alyac.co.kr', 'securityaffairs.co', 'threatpost.com', 'dragos.com', 'vb2020.vblocalhost.com', 'blogs.blackberry.com', 'vblocalhost.com', 'www.darkreading.com', 'insight-jp.nttsecurity.com', 'www.lacework.com', 'www.talent-jump.com', 'www.wired.com', 'www.accenture.com', 'cdn-cybersecurity.att.com', 'cycraft.com', 'news.sophos.com', 'blog.certfa.com', 'home.treasury.gov', 'citizenlab.ca', 'www.f-secure.com',

In [12]:
import tldextract

def extract_second_level_domain(fqdn):
    """
    Extracts the second-level domain (SLD) from a given FQDN.
    
    Args:
        fqdn (str): Fully Qualified Domain Name (FQDN).
        
    Returns:
        str: The second-level domain of the FQDN.
    """
    extracted = tldextract.extract(fqdn)
    return extracted.domain  # This gives you the second-level domain (SLD)


# Extract unique second-level domains from both lists
unique_second_level_malpedia = set(extract_second_level_domain(fqdn) for fqdn, date in malpedia_fqdn_and_dates)
unique_second_level_mitre = set(extract_second_level_domain(fqdn) for fqdn in mitre_successful_fqdns)

# Find the intersection of second-level domains
overlap = unique_second_level_malpedia.intersection(unique_second_level_mitre)
b_minus_a = unique_second_level_mitre.difference(unique_second_level_malpedia)
# Print the result
print(f"Intersection of second-level domains: {overlap}")
print(f"Domains in MITRE but not in Malpedia: {b_minus_a}")

Intersection of second-level domains: {'zscaler', 'pwc', 'crowdstrike', 'securityintelligence', 'talent-jump', 'cybleinc', 'thedfirreport', 'mandiant', 'bleepingcomputer', 'lacework', 'fireeye', 'treasury', 'us-cert', 'hubspotusercontent30', 'jpcert', 'redcanary', 'twitter', 'welivesecurity', 'ic3', 'securityaffairs', 'morphisec', 'flashpoint-intel', 'domaintools', 'cisco', 'bushidotoken', 'group-ib', 'wordpress', 'pylos', 'zdnet', 'europa', 'netzpolitik', 'rapid7', 'wired', 'mcafee', 'aquasec', 'intezer', 'ptsecurity', 'clearskysec', 'threatconnect', 'checkpoint', 'securelist', 'logrhythm', 'objective-see', 'kasperskycontenthub', 'lookout', 'talosintelligence', 'arcticwolf', 'threatpost', 'trendmicro', 'accenture', 'deepinstinct', 'broadcom', 'cyberscoop', 'thehackernews', 'ironnet', 'recordedfuture', 'alyac', 'mitre', 'volexity', 'cybercom', 'avira', 'bbc', 'forcepoint', 'dragos', 'rsa', 'nccgroup', 'therecord', 'nttsecurity', 'citizenlab', 'phishlabs', 'trustwave', 'ncsc', 'malwareb